In [1]:
import torch
from torch import nn  # NeuralNetに関するパッケージ
import torch.nn.functional as F  # 様々な関数が定義されている。as Fとしてimportするのが通例 
from torch import optim  # オプティマイザ関連
from torchvision import datasets, transforms  # 公式の画像処理ライブラリ。データのロードや変換ができる

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__() 
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=20, kernel_size=5)  # 2次元畳み込み層の定義
        self.conv2 = nn.Conv2d(in_channels=20, out_channels=50, kernel_size=5)
        self.fc1 = nn.Linear(4 * 4 * 50, 500)  # 全結合層の定義
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):  # x: (batch_size * 1 * 28 * 28)
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [3]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()  # 学習モードをオンに
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)  # GPUへの転送
        optimizer.zero_grad()  # 勾配リセット
        output = model(data)
        loss = F.nll_loss(output, target)  # 誤差計算
        loss.backward()
        optimizer.step()  # パラメータ更新
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))

In [4]:
def test(model, device, test_loader):
    model.eval()# 学習モードをオフに
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)  # 最大確率のインデックス取得
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [5]:
# 学習設定
batch_size = 64
test_batch_size=1000
epochs = 10
lr = 0.01
momentum = 0.5
no_cuda = False
seed=1
save_model = False
use_cuda = not no_cuda and torch.cuda.is_available()

torch.manual_seed(seed)
device = torch.device("cuda" if use_cuda else "cpu") #CUDA_VISIBUL

fashion_mnist_data = datasets.FashionMNIST(
    './fashion-mnist',
    transform=transforms.ToTensor(),
    download=True)
data_loader = torch.utils.data.DataLoader(
    dataset=fashion_mnist_data,
    batch_size=16,
    shuffle=True)

fashion_mnist_data_test = datasets.FashionMNIST(
    './fashion-mnist',
    transform=transforms.ToTensor(),
    train=False,
    download=True)

data_loader_test = torch.utils.data.DataLoader(
    dataset=fashion_mnist_data_test,
    batch_size=16,
    shuffle=True)

model = Net().to(device) # GPUへ転送
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

# エポックごとに学習とテスト
for epoch in range(1, epochs + 1):
    train(model, device, data_loader, optimizer, epoch)
    test(model, device, data_loader_test)

# モデル保存など
if save_model:
    torch.save(model.state_dict(),"mnist_cnn.pt")

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.307585
Train Epoch: 1 [160/60000 (0%)]	Loss: 2.317822
Train Epoch: 1 [320/60000 (1%)]	Loss: 2.280998
Train Epoch: 1 [480/60000 (1%)]	Loss: 2.277723
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.261968
Train Epoch: 1 [800/60000 (1%)]	Loss: 2.242610
Train Epoch: 1 [960/60000 (2%)]	Loss: 2.280852
Train Epoch: 1 [1120/60000 (2%)]	Loss: 2.169317
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.197433
Train Epoch: 1 [1440/60000 (2%)]	Loss: 2.109321
Train Epoch: 1 [1600/60000 (3%)]	Loss: 2.189401
Train Epoch: 1 [1760/60000 (3%)]	Loss: 2.019431
Train Epoch: 1 [1920/60000 (3%)]	Loss: 1.896518
Train Epoch: 1 [2080/60000 (3%)]	Loss: 1.844120
Train Epoch: 1 [2240/60000 (4%)]	Loss: 1.317016
Train Epoch: 1 [2400/60000 (4%)]	Loss: 1.569441
Train Epoch: 1 [2560/60000 (4%)]	Loss: 1.827097
Train Epoch: 1 [2720/60000 (5%)]	Loss: 1.183992
Train Epoch: 1 [2880/60000 (5%)]	Loss: 1.471547
Train Epoch: 1 [3040/60000 (5%)]	Loss: 0.788660
Train Epoch: 1 [3200/60000 (5%)]	Loss: 1.334535
T

Train Epoch: 1 [26560/60000 (44%)]	Loss: 0.756638
Train Epoch: 1 [26720/60000 (45%)]	Loss: 1.067206
Train Epoch: 1 [26880/60000 (45%)]	Loss: 0.716756
Train Epoch: 1 [27040/60000 (45%)]	Loss: 0.732977
Train Epoch: 1 [27200/60000 (45%)]	Loss: 0.412664
Train Epoch: 1 [27360/60000 (46%)]	Loss: 0.594655
Train Epoch: 1 [27520/60000 (46%)]	Loss: 0.521238
Train Epoch: 1 [27680/60000 (46%)]	Loss: 0.307090
Train Epoch: 1 [27840/60000 (46%)]	Loss: 0.771580
Train Epoch: 1 [28000/60000 (47%)]	Loss: 0.957828
Train Epoch: 1 [28160/60000 (47%)]	Loss: 0.520905
Train Epoch: 1 [28320/60000 (47%)]	Loss: 0.431662
Train Epoch: 1 [28480/60000 (47%)]	Loss: 0.593562
Train Epoch: 1 [28640/60000 (48%)]	Loss: 0.478042
Train Epoch: 1 [28800/60000 (48%)]	Loss: 0.404448
Train Epoch: 1 [28960/60000 (48%)]	Loss: 0.303811
Train Epoch: 1 [29120/60000 (49%)]	Loss: 0.997649
Train Epoch: 1 [29280/60000 (49%)]	Loss: 0.792620
Train Epoch: 1 [29440/60000 (49%)]	Loss: 1.366336
Train Epoch: 1 [29600/60000 (49%)]	Loss: 0.807316


Train Epoch: 1 [52640/60000 (88%)]	Loss: 0.660599
Train Epoch: 1 [52800/60000 (88%)]	Loss: 0.356758
Train Epoch: 1 [52960/60000 (88%)]	Loss: 0.592548
Train Epoch: 1 [53120/60000 (89%)]	Loss: 0.310040
Train Epoch: 1 [53280/60000 (89%)]	Loss: 0.427640
Train Epoch: 1 [53440/60000 (89%)]	Loss: 0.468995
Train Epoch: 1 [53600/60000 (89%)]	Loss: 0.505332
Train Epoch: 1 [53760/60000 (90%)]	Loss: 0.161110
Train Epoch: 1 [53920/60000 (90%)]	Loss: 0.221670
Train Epoch: 1 [54080/60000 (90%)]	Loss: 0.602221
Train Epoch: 1 [54240/60000 (90%)]	Loss: 0.662792
Train Epoch: 1 [54400/60000 (91%)]	Loss: 0.697021
Train Epoch: 1 [54560/60000 (91%)]	Loss: 0.231604
Train Epoch: 1 [54720/60000 (91%)]	Loss: 0.512400
Train Epoch: 1 [54880/60000 (91%)]	Loss: 0.688614
Train Epoch: 1 [55040/60000 (92%)]	Loss: 0.488225
Train Epoch: 1 [55200/60000 (92%)]	Loss: 0.469239
Train Epoch: 1 [55360/60000 (92%)]	Loss: 0.172468
Train Epoch: 1 [55520/60000 (93%)]	Loss: 0.560752
Train Epoch: 1 [55680/60000 (93%)]	Loss: 0.734430


Train Epoch: 2 [18880/60000 (31%)]	Loss: 0.205321
Train Epoch: 2 [19040/60000 (32%)]	Loss: 0.905396
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.294592
Train Epoch: 2 [19360/60000 (32%)]	Loss: 0.296167
Train Epoch: 2 [19520/60000 (33%)]	Loss: 0.852735
Train Epoch: 2 [19680/60000 (33%)]	Loss: 0.468929
Train Epoch: 2 [19840/60000 (33%)]	Loss: 0.736696
Train Epoch: 2 [20000/60000 (33%)]	Loss: 0.408457
Train Epoch: 2 [20160/60000 (34%)]	Loss: 0.281038
Train Epoch: 2 [20320/60000 (34%)]	Loss: 0.490400
Train Epoch: 2 [20480/60000 (34%)]	Loss: 0.259586
Train Epoch: 2 [20640/60000 (34%)]	Loss: 0.448304
Train Epoch: 2 [20800/60000 (35%)]	Loss: 0.526370
Train Epoch: 2 [20960/60000 (35%)]	Loss: 0.228301
Train Epoch: 2 [21120/60000 (35%)]	Loss: 0.544917
Train Epoch: 2 [21280/60000 (35%)]	Loss: 0.242329
Train Epoch: 2 [21440/60000 (36%)]	Loss: 0.425904
Train Epoch: 2 [21600/60000 (36%)]	Loss: 0.309290
Train Epoch: 2 [21760/60000 (36%)]	Loss: 0.839264
Train Epoch: 2 [21920/60000 (37%)]	Loss: 0.485848


Train Epoch: 2 [44960/60000 (75%)]	Loss: 0.575167
Train Epoch: 2 [45120/60000 (75%)]	Loss: 0.590812
Train Epoch: 2 [45280/60000 (75%)]	Loss: 0.508454
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.345515
Train Epoch: 2 [45600/60000 (76%)]	Loss: 0.191602
Train Epoch: 2 [45760/60000 (76%)]	Loss: 0.550218
Train Epoch: 2 [45920/60000 (77%)]	Loss: 0.387953
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.332487
Train Epoch: 2 [46240/60000 (77%)]	Loss: 0.132704
Train Epoch: 2 [46400/60000 (77%)]	Loss: 0.343632
Train Epoch: 2 [46560/60000 (78%)]	Loss: 0.286250
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.265466
Train Epoch: 2 [46880/60000 (78%)]	Loss: 0.366952
Train Epoch: 2 [47040/60000 (78%)]	Loss: 0.516887
Train Epoch: 2 [47200/60000 (79%)]	Loss: 0.400809
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.239751
Train Epoch: 2 [47520/60000 (79%)]	Loss: 0.155816
Train Epoch: 2 [47680/60000 (79%)]	Loss: 0.218112
Train Epoch: 2 [47840/60000 (80%)]	Loss: 0.692981
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.400848


...

Train Epoch: 4 [29760/60000 (50%)]	Loss: 0.194981
Train Epoch: 4 [29920/60000 (50%)]	Loss: 0.297051
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.306691
Train Epoch: 4 [30240/60000 (50%)]	Loss: 0.132220
Train Epoch: 4 [30400/60000 (51%)]	Loss: 0.365832
Train Epoch: 4 [30560/60000 (51%)]	Loss: 0.262139
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.351467
Train Epoch: 4 [30880/60000 (51%)]	Loss: 0.666977
Train Epoch: 4 [31040/60000 (52%)]	Loss: 0.128640
Train Epoch: 4 [31200/60000 (52%)]	Loss: 0.461538
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.470582
Train Epoch: 4 [31520/60000 (53%)]	Loss: 0.160712
Train Epoch: 4 [31680/60000 (53%)]	Loss: 0.519362
Train Epoch: 4 [31840/60000 (53%)]	Loss: 0.159431
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.389339
Train Epoch: 4 [32160/60000 (54%)]	Loss: 0.226525
Train Epoch: 4 [32320/60000 (54%)]	Loss: 0.172202
Train Epoch: 4 [32480/60000 (54%)]	Loss: 0.270202
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.349439
Train Epoch: 4 [32800/60000 (55%)]	Loss: 0.292240


Train Epoch: 4 [55840/60000 (93%)]	Loss: 0.037105
Train Epoch: 4 [56000/60000 (93%)]	Loss: 0.387050
Train Epoch: 4 [56160/60000 (94%)]	Loss: 0.311256
Train Epoch: 4 [56320/60000 (94%)]	Loss: 0.141621
Train Epoch: 4 [56480/60000 (94%)]	Loss: 0.070033
Train Epoch: 4 [56640/60000 (94%)]	Loss: 0.522274
Train Epoch: 4 [56800/60000 (95%)]	Loss: 0.450405
Train Epoch: 4 [56960/60000 (95%)]	Loss: 0.257396
Train Epoch: 4 [57120/60000 (95%)]	Loss: 0.258339
Train Epoch: 4 [57280/60000 (95%)]	Loss: 0.426891
Train Epoch: 4 [57440/60000 (96%)]	Loss: 0.325702
Train Epoch: 4 [57600/60000 (96%)]	Loss: 0.487821
Train Epoch: 4 [57760/60000 (96%)]	Loss: 0.566825
Train Epoch: 4 [57920/60000 (97%)]	Loss: 0.352484
Train Epoch: 4 [58080/60000 (97%)]	Loss: 0.425787
Train Epoch: 4 [58240/60000 (97%)]	Loss: 0.355992
Train Epoch: 4 [58400/60000 (97%)]	Loss: 0.366865
Train Epoch: 4 [58560/60000 (98%)]	Loss: 0.314443
Train Epoch: 4 [58720/60000 (98%)]	Loss: 0.155023
Train Epoch: 4 [58880/60000 (98%)]	Loss: 0.580144


Train Epoch: 5 [22080/60000 (37%)]	Loss: 0.143439
Train Epoch: 5 [22240/60000 (37%)]	Loss: 0.471373
Train Epoch: 5 [22400/60000 (37%)]	Loss: 0.099846
Train Epoch: 5 [22560/60000 (38%)]	Loss: 0.597058
Train Epoch: 5 [22720/60000 (38%)]	Loss: 0.197442
Train Epoch: 5 [22880/60000 (38%)]	Loss: 0.337290
Train Epoch: 5 [23040/60000 (38%)]	Loss: 0.334214
Train Epoch: 5 [23200/60000 (39%)]	Loss: 0.322967
Train Epoch: 5 [23360/60000 (39%)]	Loss: 0.214377
Train Epoch: 5 [23520/60000 (39%)]	Loss: 0.252951
Train Epoch: 5 [23680/60000 (39%)]	Loss: 0.121730
Train Epoch: 5 [23840/60000 (40%)]	Loss: 0.124222
Train Epoch: 5 [24000/60000 (40%)]	Loss: 0.813489
Train Epoch: 5 [24160/60000 (40%)]	Loss: 0.404277
Train Epoch: 5 [24320/60000 (41%)]	Loss: 0.117984
Train Epoch: 5 [24480/60000 (41%)]	Loss: 0.393296
Train Epoch: 5 [24640/60000 (41%)]	Loss: 0.337400
Train Epoch: 5 [24800/60000 (41%)]	Loss: 0.133293
Train Epoch: 5 [24960/60000 (42%)]	Loss: 0.217558
Train Epoch: 5 [25120/60000 (42%)]	Loss: 0.371442


Train Epoch: 5 [48160/60000 (80%)]	Loss: 0.123723
Train Epoch: 5 [48320/60000 (81%)]	Loss: 0.451572
Train Epoch: 5 [48480/60000 (81%)]	Loss: 0.362418
Train Epoch: 5 [48640/60000 (81%)]	Loss: 0.209586
Train Epoch: 5 [48800/60000 (81%)]	Loss: 0.492274
Train Epoch: 5 [48960/60000 (82%)]	Loss: 0.113617
Train Epoch: 5 [49120/60000 (82%)]	Loss: 0.428988
Train Epoch: 5 [49280/60000 (82%)]	Loss: 0.119726
Train Epoch: 5 [49440/60000 (82%)]	Loss: 0.494357
Train Epoch: 5 [49600/60000 (83%)]	Loss: 0.316163
Train Epoch: 5 [49760/60000 (83%)]	Loss: 0.581187
Train Epoch: 5 [49920/60000 (83%)]	Loss: 0.330008
Train Epoch: 5 [50080/60000 (83%)]	Loss: 0.386742
Train Epoch: 5 [50240/60000 (84%)]	Loss: 0.484343
Train Epoch: 5 [50400/60000 (84%)]	Loss: 0.227782
Train Epoch: 5 [50560/60000 (84%)]	Loss: 0.169353
Train Epoch: 5 [50720/60000 (85%)]	Loss: 0.227728
Train Epoch: 5 [50880/60000 (85%)]	Loss: 0.234396
Train Epoch: 5 [51040/60000 (85%)]	Loss: 0.448416
Train Epoch: 5 [51200/60000 (85%)]	Loss: 0.449025


KeyboardInterrupt: 